In [1]:
import numpy as np
import csv
import matplotlib.pyplot as plt
import pandas as pd
import os
import time
import seaborn as sns
import glob
import re
from datetime import datetime

In [2]:
class shockwave(object):
    "shockwave is represented by the initial point and slop"
    "initial point includes the coordinates of time and location"
    "initial point refers to the initial point of a shockwave within a square"
    "the range of time is from 0 to the length of time interval"
    "the location is from 0 to the length of the road section"
    
    def __init__(self, s1, s2, ini_t, x, t_e):
        "s1 is the traffic state of the upstream traffic state and s2 is the downstream one"
        "ini_t is the start time of this shockwave, x is the start position of this shockwave"
        "t_e is the end time of this interval"
        self.uk = s1[0] # upstream traffic density
        self.dk = s2[0] # downstream traffic density
        self.uq = s1[1] # upstream flow
        self.dq = s2[1] # downstream flow
        if self.uq == self.dq or self.uk == self.dk:
            self.slope = 1 / 10000000000000000000000000000

        else:
            self.slope = (self.uq - self.dq) / (self.uk - self.dk) # shockwave slope
        self.ini_t = ini_t # initial coordinate on time axis
        self.ini_x = x # initial coordinate on space axis
        self.end_t = t_e # the end coordinate on time axis
        self.end_x = self.ini_x + (self.end_t - self.ini_t) * self.slope # the end coordinate on space axis

    def spatialLocation (t): # to calculate the position of the shockwave given a time
        return (self.ini_x + (t - self.ini_t) * self.slope)

In [3]:
# This function is to calculate the slope of the shockwave between two neighboring traffic states.
def sw_slope(s1, s2): # s1 and s1 are in the form of [k, q]
    slope = (s1[1] - s2[1]) / (s1[0] - s2[0])
    return slope

In [4]:
# To indicate the properties of shockwaves
def shockwave_property (sw): # to print the properties of a shockwave, the input of which is a shockwave object.
    print ('up state = ', [sw.uk, sw.uq])
    print ('down state = ', [sw.dk, sw.dq])
    print ('slope = ', sw.slope)
    print ('initial point = ', [sw.ini_t, sw.ini_x])
    print ('end point = ', [sw.end_t, sw.end_x])

def sw_List_property (swl): # to print the properties of a shockwave, the input of which is a list of shockwave object.
    for i in swl:
        print ('the properties of a shockwave:')
        shockwave_property (i)
        print ('')

In [5]:
'Modifying this function'
# This function is to convert the traffic state list and their boundaries positions to a list of shockwaves in the form of object,
# and to convert the traffic state list and their boundaries positions to a list of state propagating into the space in list
###############
# The initialization can generate the shockwave list including the boundary conditions induced ones
# The initialization can also generate the traffic state list on this road section starting from the generation the shockwave list 
###############

def Initialization (us, ds, s, swp, t_0, t_e, L): # the output of this function is a list of shockwaves which is in the form of object.
    "us is the upstream boundary condition, which is in the form of [k, q]"
    "ds is the downstream boundary condition, which is in the form of [k, q]"
    "s is the list of traffic state on the road at the start of a sub-interval, the entry of which is in the form of [k, q]"
    "swp is the list of shockwave position on the road at t_0, the entry of which is the position of the boundary between two neighboring states"
    "t_0 is the start point of this sub time interval, which may be the start of this time interval"
    "t_e is the end point of this time interval"
    "L is the length of this road section"
    swl = []
    sl = s
    if len (s) <= 0:
        print ('There is no traffic state. Please input it!')

    elif len (s) == 1:
        if s[0][0] == us[0] == ds[0]:
            swl = []
        if s[0][0] != us[0]:
            slope_u = (s[0][1] - us[1])/(s[0][0] - us[0])
            if slope_u > 0:
                sw_u = shockwave(us, s[0], t_0, 0, t_e)
                swl.append(sw_u)
                sl = [us] + sl
                
    else:
        if s[0][0] != us[0]:
            slope_u = (s[0][1] - us[1])/(s[0][0] - us[0])
            
            if slope_u > 0:
                sw_u = shockwave(us, s[0], t_0, 0, t_e)
                swl.append(sw_u)
                sl = [us] + sl

        if len (s) > 1:
            for i in range (len(s) - 1):
                swl.append(shockwave(s[i], s[i+1], t_0, swp[i], t_e))

        if s[-1][0] != ds[0]:
            slope_d = (s[-1][1] - ds[1])/(s[-1][0] - ds[0])
            
            if slope_d < 0:
                ini_d = [t_0, L]
                sw_d = shockwave(s[-1], ds, t_0, L, t_e)
                swl.append(sw_d) 
                sl = sl + [ds]
    slops = slop_generation (swl) # the slops of all the shockwaves
    start_points = sw_startPosition_generation (swl) # the start points of all the shockwaves
    return swl, sl, slops, start_points

In [6]:
# These functions are to list out the shockwave slopes, start positions, end posiitons, and states in the shockwave list
def slop_generation (swl): # The input is the list of shockwave.
    if len(swl) <= 0:
        #print ('There is no shockwave.')
        slope = []
        
    if len(swl) > 0:
        slope = []
        for i in swl:
            slope.append (i.slope)
    return slope

def sw_startPosition_generation (swl): # The input is the list of shockwave.
    iniP = []
        # print ('There is no shockwave.')
    if len(swl) > 0:
        iniP = []
        for i in swl:
            iniP.append (i.ini_x)
    return iniP

def sw_endPosition_generation (swl): # The input is the list of shockwave.
    endP = []
    # if len(swl) <= 0:
    #     print ('There is no shockwave.')
        
    if len(swl) > 0:
        
        for i in swl:
            endP.append (i.end_x)
        endP=sorted(endP)
    return endP

In [7]:
# to calculate the intersection of two shockwaves
def intersection(s1, s2):
    int_t = (s1.slope * s1.ini_t - s2.slope * s2.ini_t + s2.ini_x - s1.ini_x) / (s1.slope - s2.slope)
    int_x = s1.slope * (int_t - s1.ini_t) + s1.ini_x
    inter_coor = [int_t, int_x] # the coordinates of the intersection
    return inter_coor
# to identify if this is in this square

# the intersection between a shockwave and a spatial boundary
def bdInsc (sw, t_0, t_e, L): 
    if sw.slope == 0:
        print ('slope is zero')
    else:
        if sw.slope < 0:
            t = - sw.ini_x / sw.slope + sw.ini_t
            p = [t, 0]
        else:
            t = (L - sw.ini_x) / sw.slope + sw.ini_t
            p = [t, L]
        return p
        # print ("slope is equal to 0")

def intrsctPositionIdentify (inter_coor, t_0, t_e, L):
    if t_e >= inter_coor[0] >= t_0:
        if inter_coor[1] >= 0:
            return 1
        else:
            return 0
    else:
        return 0

In [8]:
# To calculate the earliest intersection
def earlistIntersection(swl, t_0, t_e, L):
    intsc = [] # the index of shockwave and intersection position
    min_t = t_e # the earliest intersection time
    min_index = -4 # the initialized value for the index of the minimum shockwave, -4 refers to an initial index which is not found.
    position = [-1, -1]  # the initialized value for the index of the minimum shockwave
    if len(swl) > 0:
        if len(swl) == 1:
            p = bdInsc(swl[0], t_0, t_e, L) # the intersection between the shockwave and a boundary
            if intrsctPositionIdentify (p, t_0, t_e, L):
                if swl[0].slope < 0:
                    min_index = -3.3 # backward shockwave

                else:
                    min_index = -3.7 # forward shockwave
                # min_index = -3 # -3 refers to that there is only one shockwave, which will intersect a boundary within the current interval
                position = p
        else:
            if swl[0].slope < 0:
                p = bdInsc(swl[0], t_0, t_e, L)
                if intrsctPositionIdentify (p, t_0, t_e, L):
                    if p[0] < min_t:
                        min_t = p[0]
                        min_index = -1 # -1 refers to that the first shockwave will intersect the upstream boundary
                        position = p
            if swl[-1].slope > 0:
                p = bdInsc(swl[-1], t_0, t_e, L)
                if intrsctPositionIdentify (p, t_0, t_e, L):
                    if p[0] < min_t:
                        min_t = p[0]
                        min_index = -2 # -2 refers to the last shockwave will intersect the downstream boundary
                        position = p
            for i in range(len(swl)-1):
                if swl[i].slope > swl[i+1].slope:
                    p = intersection (swl[i], swl[i+1]) # position is the positioin of intersection
                    if intrsctPositionIdentify (p, t_0, t_e, L):
                        if p[0] < min_t:
                            min_t = p[0]
                            min_index = i # the intersection between the ith shockwave and the i+1th shockwave
                            position = p
    return [min_index, position] # the first is the index of the minmum one and the second one is its coordinates

In [9]:
# This function is to generate state list on the road at the earliest intersection
# This function builds up the relationship between the state list within a sub-interval and that at the time of the earlist intersection.
def state_update (earlistIntersection, sl): # sl is the traffic states within a sub interval
    # s is the traffic state at the time of the earliest intersection
    if len(sl) > 1:
        if earlistIntersection[0] == -3.3: # just one backward shockwave on the temporal-spatial area
            s = [sl[-1]]
        elif earlistIntersection[0] == -3.7: # just one forward shockwave on the temporal-spatial area
            s = [sl[0]]
    
        elif earlistIntersection[0] == -2: #-2 refers to the last shockwave will intersect the downstream boundary
            s = sl[:len(sl)-1]   #maybe have bug
            
        elif earlistIntersection[0] == -1: # -1 refers to that the first shockwave will intersect the upstream boundary
            s = sl[1:]
        else:
            min_index = earlistIntersection[0]
            s = sl[:min_index + 1] + sl[min_index + 2:]

    else:
        s = sl

    return s

In [10]:
# This function is to generate the shockwave position at the end of a sub interval
# This function builds up the relationship between the shockwave list within a sub-interval and that at the time of the earlist intersection.
def swp_update (earlistIntersection, swl): #swl is the shockwave list within a sub-interval
    swp = []
    # print (earlistIntersection[0])
    if len(swl) > 0:
        t_e = earlistIntersection[1][0] # earlistIntersection[1][0] is the time of earliest intersection
        if -4 < earlistIntersection[0] < -3: # There is only one shockwave in the shockwave list
            swp = []
        elif earlistIntersection[0] == -2:
            "drop the last shockwave and keep the others"
            
            sw_Lst = swl[:len(swl)-1]
            for i in sw_Lst:
                swp.append(i.ini_x + (t_e - i.ini_t) * i.slope) 
        elif earlistIntersection[0] == -1:
            sw_Lst = swl[1:]
            for i in sw_Lst:
                swp.append(i.ini_x + (t_e - i.ini_t) * i.slope) 
        else:
            min_index = earlistIntersection[0]
            for i in swl[:min_index]:
                swp.append(i.ini_x + (t_e - i.ini_t) * i.slope) 
            for i in swl[min_index + 1:]:
                swp.append(i.ini_x + (t_e - i.ini_t) * i.slope)
                
            swp=sorted(swp)  # maybe need debug

    return swp   # maybe also need to return sw_Lst

In [11]:
# This function is to generate the final result of a shockwave graph in an interval
def swg1 (us, ds, s, swp, t_0, t_e, L):
    "s is he initial state at the start of an interval"
    "swp is the initial shockwave positions at the start of an interval"
    s_ini = s
    t_ini = t_0
    State = [] # The list of traffic state at every sub-interval, the entry of which is a list of state.
    State_ini = [] # the initial state
    ShockwavePosition = [] # The list of shockwave start positions at every sub-interval, the entry of which is a list of positions.
    Slopes = [] # The list of the slopes of all shockwave at every sub-interval.
    TS = [] #The time series of sub-interval
    Inter_P=[] # store the intersection point of shockwaves within the interval 
    EarlyIndex = -5 # To activate this recusion to show that it is not equal to -4
    count=0
    while EarlyIndex != -4:
        sw_Lst, s_Lst, slp_Lst, swp_Lst = Initialization (us, ds, s_ini, swp, t_ini, t_e, L)
        count=count+1
        print("debug:",count)
        "sw_Lst is the shockwave list"
        "s_Lst is the state list"
        "slp_Lst is the slope list"
        "swp_Lst is the shockwave start position list"
        # To generate the list of shockwave slopes.
        # slps = slop_generation (sw_List) # The slopes of all the shockwaves
        # for i in sw_List:
        #     slps.append(i.slope)
        Slopes.append(slp_Lst)
        #swp = sw_startPosition_generation (swl) # The start points of all the shockwaves
        ShockwavePosition.append(swp_Lst)
        TS.append(t_ini)
        insc = earlistIntersection(sw_Lst, t_ini, t_e, L)
        # print ('insc = ', insc)
        EarlyIndex = insc[0]
        #print (EarlyIndex)
        State.append(s_Lst)
        # ite = 1
        # print ('ite = ', ite)
        if EarlyIndex != -4:
            # ite += 1
            s_ini = state_update (insc, s_Lst)
            swp = swp_update (insc, sw_Lst)
            t_ini = insc[1][0]   # the time of earliest intersection.
            Inter_P.append(insc[1])
            #print("intersections within the interval", insc[1])
            # ShockwavePosition.append(swp) # the positions of shockwaves at earliest intersection.
            
        else: 
            "This needs to debug.XxxxxXXXXXXXX"   
            sw_endP = sw_endPosition_generation (sw_Lst)  # final sw (in the latest subinterval) end points
            s_endP = State[-1] # final state list in the latest subinterval 
            TS.append(t_e)

    return State, ShockwavePosition, Slopes, TS, sw_Lst, sw_endP, s_endP, Inter_P  # sw_Lst is the final shockwave list

In [12]:
def state2matrix (ShockwavePosition, Slopes, TS, State, L, t_0, t_invl, N_row, N_column): 
    # L is the length of the road, t_invl is the time interval
    # N_row is the temporal resolution
    # N_column is the column resolution

    # The position in the matrix
    delt_t = t_invl / N_row
    delt_l = L / N_column
    
    T = [] # time list
    for i in range (N_row):
        T.append(t_0+(i+0.5) * delt_t)
        #T.append((i+0.5) * delt_t)  # The position of detect point temporal position
    P = [] # position list
    for i in range (N_column):   
        P.append((i+0.5) * delt_l) # The position of detect point spatial position

    
    TIdx = []
    legth = len(TS)
    for i in range (N_row):
        for j in range (legth-1): 
            if TS[j] < T[i] <= TS[j+1]: # This time  belongs to the h^th to the h+1^th time range
                TIdx.append(j)
    
    SPM = [] # The position of shockwave at each time point of the temporal-spatial region, which is a matrix to represent all shockwave points.
    for i in range (len(T)):
        shockP = [] # The positions of shockwave at a time point , which is a list to represent all shockwave points.
        for j in range (len(Slopes[TIdx[i]])):
            shockP.append((T[i] - TS[TIdx[i]]) * Slopes[TIdx[i]][j] + ShockwavePosition[TIdx[i]][j])
        SPM.append(shockP)
    
    SIdx = [] # the matrix to indicate the index of the spatial point at every time point.
    for i in range (N_row):
        positonRange = [0] + SPM[i] + [L]
        sIdx = [] # the matrix to indicate the index of the spatial point at a time point.
        for k in range (N_column):
            for j in range (len (positonRange) - 1):
                if positonRange[j] < P[k] <= positonRange[j+1]:
                    sIdx.append (j)
        SIdx.append(sIdx)
    
    StateMatrix = [] # The traffic state in matrix
    for i in range (N_row):
        Statelst = [] # The traffic state in matrix
        for j in SIdx[i]:
            Statelst.append (State[TIdx[i]][j])
        StateMatrix.append (Statelst)
    return StateMatrix

# test example for single SW diagram encoding start

In [13]:
t_0 = 0.00   # the unit is hour
t_e = 0.01   # the unit is hour
L = 1 # mile
us = [40, 2400]
ds = [150, 900]
s_ini = [[50, 3000], [40, 2400], [30, 1800]]
swp = [0.4, 0.7]  # the unit is mile
State, ShockwavePosition, Slopes, TS, sw_Lst, sw_endP, s_endP, Inter_P= swg1 (us, ds, s_ini, swp, t_0, t_e, L)

debug: 1
debug: 2
debug: 3


In [14]:
print("--------State--------")
print(State)
print("--------ShockwavePosition--------")
print(ShockwavePosition)
print("--------Slopes--------")
print(Slopes)
print("--------TS--------")
print(TS)

--------State--------
[[[40, 2400], [50, 3000], [40, 2400], [30, 1800], [150, 900]], [[40, 2400], [50, 3000], [40, 2400], [150, 900]], [[40, 2400], [50, 3000], [150, 900]]]
--------ShockwavePosition--------
[[0, 0.4, 0.7, 1], [0.2666666666666667, 0.6666666666666667, 0.9666666666666667], [0.5111111111111111, 0.9111111111111111]]
--------Slopes--------
[[60.0, 60.0, 60.0, -7.5], [60.0, 60.0, -13.636363636363637], [60.0, -21.0]]
--------TS--------
[0.0, 0.004444444444444445, 0.008518518518518517, 0.01]


In [16]:
# # Traffic state matric setput
N_row = 6
N_column = 4
t_invl = 0.01
matrix_test=state2matrix(ShockwavePosition, Slopes, TS, State, L, t_0, t_invl, N_row, N_column)


In [19]:
# matrix_den=np.zeros((N_column,N_row))
matrix_vol=np.zeros((N_column,N_row))
print(N_column,N_row)
matrix_den
for i in range(N_row):
    for j in range(N_column):
        #print(i,j)
        matrix_den[N_column-1-j][i]=matrix_test[i][j][0]
        matrix_vol[N_column-1-j][i]=matrix_test[i][j][1]
print(matrix_den)
print(matrix_vol)

4 6
[[30. 30. 40. 40. 40. 50.]
 [40. 40. 50. 50. 50. 50.]
 [50. 50. 50. 50. 40. 40.]
 [50. 40. 40. 40. 40. 40.]]
[[1800. 1800. 2400. 2400. 2400. 3000.]
 [2400. 2400. 3000. 3000. 3000. 3000.]
 [3000. 3000. 3000. 3000. 2400. 2400.]
 [3000. 2400. 2400. 2400. 2400. 2400.]]


# test end

In [20]:
def flatten_and_separate(input_list):
    # Initialize two lists to hold the separated values
    input_array=np.array(input_list)
    N_column=input_array.shape[1]
    N_row=input_array.shape[0]
    
    matrix_den=np.zeros((N_column,N_row))
    matrix_vol=np.zeros((N_column,N_row))

    for i in range(N_row):
        for j in range(N_column):
            matrix_den[N_column-1-j][i]=input_list[i][j][0]
            matrix_vol[N_column-1-j][i]=input_list[i][j][1]
            
    denlist = matrix_den.flatten().tolist()
    vollist = matrix_vol.flatten().tolist()

    return denlist, vollist


# Load traffic density and volume data

In [22]:
den_2947 = pd.read_csv('concatenated_density_2947_month_5to9.csv')
den_2958 = pd.read_csv('concatenated_density_2958_month_5to9.csv')
den_998 = pd.read_csv('concatenated_density_998_month_5to9.csv')

vol_2947 = pd.read_csv('concatenated_volume_2947_month_5to9.csv')
vol_2958 = pd.read_csv('concatenated_volume_2958_month_5to9.csv')
vol_998 = pd.read_csv('concatenated_volume_998_month_5to9.csv')

den_2947=den_2947['0'].tolist()
den_2958=den_2958['0'].tolist()
den_998=den_998['0'].tolist()
vol_2947=vol_2947['0'].tolist()
vol_2958=vol_2958['0'].tolist()
vol_998=vol_998['0'].tolist()
print(len(den_2947),len(den_2958),len(den_998),len(vol_2947),len(vol_2947),len(vol_998))

# generate SW diagrams' pixel based encoding results

In [ ]:
# SW diagram between ramp 2947 to ramp 2958
# code below is for pixel under 9*12 resolution, 9*6 for 2947-2958 and another 9*6 for 2958-998. N_row and N_column can be set for other resolution, 
from IPython.display import clear_output

# pixel resolutions
N_row = 9      # 3,6,9,12
N_column = 6   # 2,4,6,8
t_invl = 20/3600
running_time=0

for I in range(150):
    clear_output(wait=True)
    df_flatten_pixel = pd.DataFrame()
    print(f"The code running time is {running_time} seconds.")
    start_time = time.time()
    for ii in range(4320*I,4320*(I+1)):  # MAY 0-133920, JUNE 133920-263520, JULY 263520-397440, AUGUST 397440-531360, 531360-660960
        print("DAY",I, "interation:",ii)
        #print("vol veh/h:", UPstream["volume"][ii]*120)
        #print("den veh/mile:", UPstream["occupancy"][ii]*5280/20*0.01)  # 5280 ft=1mile, 20ft=length of car+length of detector
        #print("vol veh/h:", Downstream["volume"][ii]*120)
        #print("den veh/mile:", Downstream["occupancy"][ii]*5280/20*0.01)  # 5280 ft=1mile, 20ft=length of car+length of detector
        t_0=ii*20/3600  # unit:hour
        t_e=t_0+20/3600 # unit:hour
        L=0.49           # unit:mile
        us=[den_2947[ii], vol_2947[ii]*180]      #[veh/mile, veh/h]
        ds=[den_2958[ii], vol_2958[ii]*180]      #[veh/mile, veh/h]
        print("us:",us, "ds:",ds)

        if ii==0:
            s_ini=[[2.1,120],[2.765,180]]
            swp=[0.2]
        elif ii==518399:
            s_ini=[[2.379, 180], [4.941, 360]]
            swp=[0.3903200624455301]  
        elif ii==531360:
            s_ini=[[0.0, 0], [5.856, 360]]
            swp=[0.3415300546398388]
        elif ii==585741:
            s_ini=[[25.254, 1080], [45.384, 1440], [28.914, 1260], [77.592, 1440]]
            swp=[0.09935419771340767, 0.15143290831638716, 0.26689944119610437]
        elif ii==428762:   # bug location
            s_ini=[[21.228, 1080], [14.457, 720], [8.784, 540]]
            swp=[0.29537734455337405, 0.3625471531766078]
        elif ii==494951:   # bug location
            s_ini=[[63.318, 1440], [68.991, 1800], [31.659, 540], [17.019, 720]]
            swp=[0.35254715317660773, 0.37501339333001915, 0.41983606556617314]
        else:
            s_ini=[]
            s_ini=s_ini+s_endP  # need to change
            swp=[]
            swp=swp+sw_endP

        for h in range(len(swp)):   # maybe I need to change the dataframe based on this
            if 0.489999999<swp[h]<=0.49000001:
                swp[h]=0.489    # such setting may result in bugs
            if -0.00000001<swp[h]<0.00000001:
                swp[h]=0.001    # such setting may result in bugs

        # -----------
        differences = [swp[i+1] - swp[i] for i in range(len(swp)-1)]
        indices_of_0 = [i for i, x in enumerate(differences) if np.abs(x)<0.00000001]
        indices_of_0_reversed=sorted(indices_of_0, reverse=True)
        for index in indices_of_0_reversed:
            del swp[index]
            del s_ini[index+1]    # maybe I need to change the dataframe based on this
        # -----------

        print("s_ini", s_ini)
        print("swp",swp)
        State, ShockwavePosition, Slopes, TS, sw_Lst, sw_endP, s_endP, Inter_P= swg1 (us, ds, s_ini, swp, t_0, t_e, L)
        matrix_temp=state2matrix(ShockwavePosition, Slopes, TS, State, L, t_0, t_invl, N_row, N_column)
        list_den, list_vol = flatten_and_separate(matrix_temp)
        if len(list_den)>N_row*N_column or len(list_vol)>N_row*N_column:
            print("debug this line")
            break
        row_temp=[list_den+list_vol]
        df_flatten_pixel = pd.concat([df_flatten_pixel, pd.DataFrame(row_temp)],axis=0, ignore_index=True)

    end_time = time.time()
    running_time = end_time - start_time
    
    # save one day's encoding results to a file

    if len(df_flatten_pixel)==4320:
        filename = f"Pixel_47to58_{I}.csv"
        df_flatten_pixel.to_csv(filename, index=False)
    else:
        print("DEBUG")
        break
        

In [ ]:
# Define the path pattern for the CSV files
path_pattern = 'Pixel_47to58_*.csv'

# Use glob to find all files matching the pattern
file_paths = glob.glob(path_pattern)

# Sort the file paths based on the numerical part of the filename
sorted_file_paths = sorted(file_paths, key=lambda x: int(re.search(r'(\d+)\.csv$', x).group(1)))

# Read each file into a DataFrame and store in a list
dataframes = [pd.read_csv(file) for file in sorted_file_paths]

# Concatenate all DataFrames along axis 0
concatenated_df = pd.concat(dataframes, axis=0)

# Optionally, save the concatenated DataFrame to a new CSV file
output_file_path = 'Pixel_9by12_47to58_concatenated.csv'
concatenated_df.to_csv(output_file_path, index=False)

print(f"All files have been concatenated and saved to {output_file_path}")


In [ ]:
check_na = pd.read_csv('Pixel_9by12_47to58_concatenated.csv')
check_na

In [ ]:
# SW diagram between ramp 2958 to ramp 998
from IPython.display import clear_output

N_row = 9      # 3,6,9,12,18
N_column = 6   # 2,4,6,8,12
t_invl = 20/3600
running_time=0
Bug_flag=0

for I in range(150):
    clear_output(wait=True)
    df_flatten_pixel = pd.DataFrame()
    print(f"The code running time is {running_time} seconds.")
    start_time = time.time()
    for ii in range(4320*I,4320*(I+1)):  # MAY 0-133920, JUNE 133920-263520, JULY 263520-397440, AUGUST 397440-531360, 531360-660960
        print("DAY",I, "interation:",ii)
        #print("vol veh/h:", UPstream["volume"][ii]*120)
        #print("den veh/mile:", UPstream["occupancy"][ii]*5280/20*0.01)  # 5280 ft=1mile, 20ft=length of car+length of detector
        #print("vol veh/h:", Downstream["volume"][ii]*120)
        #print("den veh/mile:", Downstream["occupancy"][ii]*5280/20*0.01)  # 5280 ft=1mile, 20ft=length of car+length of detector
        t_0=ii*20/3600  # unit:hour
        t_e=t_0+20/3600 # unit:hour
        L=0.4           # unit:mile
        us=[den_2958[ii], vol_2958[ii]*180]      #[veh/mile, veh/h]
        ds=[den_998[ii], vol_998[ii]*180]      #[veh/mile, veh/h]
        print("us:",us, "ds:",ds)

        if ii==0:
            s_ini=[[2.765, 180], [0.0, 0]]
            swp=[0.203200624455301]
        elif ii==286639:
            s_ini=[[144.753, 720], [50.691, 540], [84.546, 1080], [97.722, 1260]]
            swp=[0.085050285980349, 0.3898404962210432, 0.399]
        elif ii==41256:
            s_ini=[[27.084, 1080], [6.222, 360], [31.293, 900]]
            swp=[0.19173617102881713, 0.24932033026224625]
        else:
            s_ini=[]
            s_ini=s_ini+s_endP  # need to change
            swp=[]
            swp=swp+sw_endP

        for h in range(len(swp)):   # maybe I need to change the dataframe based on this
            if 0.399999999<swp[h]<=0.41000001:   #0.40000001
                swp[h]=0.399    # such setting may result in bugs
            if -0.00000001<swp[h]<0.00000001:
                swp[h]=0.001    # such setting may result in bugs
            if swp[h]>0.41000001:
                print("DEBUG!!!!!")
                Bug_flag=1
                break
                
        if Bug_flag==1:
            print("HAHA BUG")
            break
                          
        # -----------
        differences = [swp[i+1] - swp[i] for i in range(len(swp)-1)]
        indices_of_0 = [i for i, x in enumerate(differences) if np.abs(x)<0.00000001]
        indices_of_0_reversed=sorted(indices_of_0, reverse=True)
        for index in indices_of_0_reversed:
            del swp[index]
            del s_ini[index+1]    # maybe I need to change the dataframe based on this
        # -----------

        print("s_ini", s_ini)
        print("swp",swp)
        State, ShockwavePosition, Slopes, TS, sw_Lst, sw_endP, s_endP, Inter_P= swg1 (us, ds, s_ini, swp, t_0, t_e, L)
        matrix_temp=state2matrix(ShockwavePosition, Slopes, TS, State, L, t_0, t_invl, N_row, N_column)
        list_den, list_vol = flatten_and_separate(matrix_temp)
        if len(list_den)>N_row*N_column or len(list_vol)>N_row*N_column:
            print("debug this line")
            break
        row_temp=[list_den+list_vol]
        df_flatten_pixel = pd.concat([df_flatten_pixel, pd.DataFrame(row_temp)],axis=0, ignore_index=True)

    end_time = time.time()
    running_time = end_time - start_time
    

    if len(df_flatten_pixel)==4320:
        filename = f"Pixel_58to98_{I}.csv"
        df_flatten_pixel.to_csv(filename, index=False)
    else:
        print("DEBUG")
        break
        

In [ ]:
# Define the path pattern for the CSV files
path_pattern = 'Pixel_58to98_*.csv'

# Use glob to find all files matching the pattern
file_paths = glob.glob(path_pattern)

# Sort the file paths based on the numerical part of the filename
sorted_file_paths = sorted(file_paths, key=lambda x: int(re.search(r'(\d+)\.csv$', x).group(1)))

# Read each file into a DataFrame and store in a list
dataframes = [pd.read_csv(file) for file in sorted_file_paths]

# Concatenate all DataFrames along axis 0
concatenated_df = pd.concat(dataframes, axis=0)

# Optionally, save the concatenated DataFrame to a new CSV file
output_file_path = 'Pixel_9by12_58to98_concatenated.csv'
concatenated_df.to_csv(output_file_path, index=False)

print(f"All files have been concatenated and saved to {output_file_path}")


In [ ]:
check_na = pd.read_csv('Pixel_9by12_58to98_concatenated.csv')
check_na

In [ ]:
pixel_58_98 = pd.read_csv('Pixel_9by12_58to98_concatenated.csv')

copied_58_98 = pixel_58_98.copy()
check_df1 = copied_58_98[:4320]

#-------------------------------------------------------------------------------------------------------
pixel_47_58 = pd.read_csv('Pixel_9by12_47to58_concatenated.csv')

# Generate new column names with 'a' appended
new_column_names = {str(col): str(col) + 'a' for col in pixel_47_58.columns}
# Rename the columns
pixel_47_58.rename(columns=new_column_names, inplace=True)

check_df2a = pixel_47_58[:4320]

#-----------------------------------------------------------------------------------------------------
concat_pixel = pd.concat([pixel_58_98.iloc[:, :9*12], pixel_47_58.iloc[:, :9*12]], axis=1)
check_df_final=concat_pixel[:4320]

concat_pixel.to_csv("Pixel_9by12_47to98_concatenated.csv", index=False)

